In [1]:
import sqlite3
import os
import datetime as dt
from datetime import date, datetime

import pandas as pd
import numpy as np

import tqdm

from tqdm.notebook import tqdm

In [2]:
# Connecting to the FYP database
connection = sqlite3.connect(os.path.join(os.getcwd(), "../Database/fypdb-PnF.sqlite"))

# Creating a cursor object to execute SQL queries on a database table
cursor = connection.cursor()

In [3]:
df = pd.read_sql_query("SELECT * from stocks", connection)

df.head()

,category,ticker,stockDate,open,close,pre_market,afterHours,high,low
0,Pharma,PFE,2020-05-13,37.3100,37.05,37.095,37.15,37.4250,36.585
1,Pharma,AZN,2020-05-13,54.8200,55.31,54.920,55.45,55.5901,54.590
2,Pharma,MRNA,2020-05-13,63.8003,65.18,63.350,65.18,66.9700,60.290
3,Pharma,PFE,2020-05-14,36.8000,37.92,37.490,38.07,37.9600,36.780
4,Pharma,AZN,2020-05-14,53.9900,53.50,54.730,53.58,54.4600,53.200


In [4]:
g_df = df.groupby('ticker')

In [5]:
def approximate_values(open_date_map, close_date_map, high_date_map, low_date_map, existing_dates):
    starting_date = existing_dates[0]

    for i in range(1, len(existing_dates)):
        cur_date = existing_dates[i]

        while starting_date + dt.timedelta(days = 1) < cur_date:
            str_start_date, str_cur_date = starting_date.strftime('%Y-%m-%d'), cur_date.strftime('%Y-%m-%d')
            
            next_date = starting_date + dt.timedelta(days = 1)
            
            str_next_date = next_date.strftime('%Y-%m-%d')

            start_open, start_close = open_date_map[str_start_date], close_date_map[str_start_date]
            end_open, end_close = open_date_map[str_cur_date], close_date_map[str_cur_date]

            start_high, start_low = high_date_map[str_start_date], low_date_map[str_start_date]
            end_high, end_low = high_date_map[str_cur_date], low_date_map[str_cur_date]

            next_open = (start_open + end_open) / 2
            next_close = (start_close + end_close) / 2
            
            next_high = (start_high + end_high) / 2
            next_low = (start_low + end_low) / 2
            
            open_date_map[str_next_date] = next_open
            close_date_map[str_next_date] = next_close
            high_date_map[str_next_date] = next_high
            low_date_map[str_next_date] = next_low

            starting_date = next_date

        starting_date = cur_date
        
        
    open_date_map = dict(sorted(open_date_map.items()))
    close_date_map = dict(sorted(close_date_map.items()))
    high_date_map = dict(sorted(high_date_map.items()))
    low_date_map = dict(sorted(low_date_map.items()))
        
    return open_date_map, close_date_map, high_date_map, low_date_map

In [6]:
# Connecting to the FYP database
connection2 = sqlite3.connect(os.path.join(os.getcwd(), "../Database/fypdb-stock.sqlite"))

# Creating a cursor object to execute SQL queries on a database table
cursor2 = connection2.cursor()

In [7]:
def create_table(cursor):
    
    drop_table = f'''
            DROP TABLE IF EXISTS stock_data;
            '''

    #cursor.execute(drop_table)
    
    create_table = '''CREATE TABLE IF NOT EXISTS stock_data(
                        category TEXT,
                        ticker TEXT,
                        stockDate DATE,
                        open REAL,
                        close REAL,
                        high REAL,
                        low REAL
                        );
                    '''
    
    cursor.execute(create_table)

    
create_table(cursor2)

In [8]:
def write_to_db(cursor, connection, category, ticker, open_date_map, close_date_map, high_date_map, low_date_map):
    
    contents = []
    
    for date in open_date_map.keys():
        contents.append((category, ticker, date, open_date_map[date], close_date_map[date], high_date_map[date], low_date_map[date]))
    
    insert_records = "INSERT INTO stock_data (category, ticker, stockDate, open, close, high, low) VALUES(?, ?, ?, ?, ?, ?, ?)"
    cursor.executemany(insert_records, contents)
    
    connection.commit()

In [9]:
progress_bar = tqdm(enumerate(df.ticker.unique()))

print(df.ticker.unique())

for i, ticker in progress_bar:
    temp_df = g_df.get_group(ticker)
    
    category = temp_df['category'].unique()[0]
    
    data_dict_list = temp_df.to_dict('records')
    existing_dates = temp_df['stockDate'].to_list()
    
    existing_dates = [datetime.strptime(date, '%Y-%m-%d') for date in existing_dates]
    
    sorted(existing_dates)
    open_date_map = dict(zip(temp_df.stockDate, temp_df.open))
    close_date_map = dict(zip(temp_df.stockDate, temp_df.close))
    high_date_map = dict(zip(temp_df.stockDate, temp_df.high))
    low_date_map = dict(zip(temp_df.stockDate, temp_df.low))

    open_date_map, close_date_map, high_date_map, low_date_map = approximate_values(open_date_map, close_date_map, high_date_map, low_date_map, existing_dates)
    
    write_to_db(cursor2, connection2, category, ticker, open_date_map, close_date_map, high_date_map, low_date_map)
    
    progress_bar.set_description(f'Working on {ticker}...')

0it [00:00, ?it/s]

['PFE' 'AZN' 'MRNA' 'MCD' 'MDLZ' 'PEP']
